<img src='./figures/logo-ecole-polytechnique-ve.jpg' style='position:absolute; top:0; right:0;' width='100px' height='' alt='' />

<center><b>Bachelor of Ecole Polytechnique</b></center>
<center>MAA106: Introduction to Numerical Analysis, year 1, semester 2</center>
<center>Authors: Zhihui Li and Kevin Messali</center>
<br>
<center><i>Approximation of periodic functions</i></center>

# Approximation of periodic functions

&nbsp;

In [41]:
import numpy as np
import scipy.integrate as integrate

In [36]:
"""
Definition of f: f is a 2-pi periodic even function on (-pi, pi] defined such that f(x) = 1 if x belongs to [-pi/2, pi/2] and f(x) = -1 otherwise.
"""

def an_f1(n):
    """
    We get a convergence for the coefficient a_n as n tends to infinity to 0 by trigonometric limits of the form sin(x)/x as x tends to infinity.
    """
    return 2*np.sin(n*np.pi/2)/(n*np.pi)

def bn_f1(n):
    """
    Since f is an even function, by remark 1.2 b_n is 0.
    """
    return 0

n = 5
print(an_f(n)) # Convergent to 0
print(bn_f(n))  # Getting 0 as expected

0.1273239544735162
0.0


In [40]:
"""
Definition of g: g is a 2-pi periodic even function on (-pi, pi] defined such that g(x) = 1 - (2 * abs(x)/pi)
"""
def an_f(n):
    """
    We get a convergence for the coefficient a_n as n tends to infinity to 0. 
    """
    return ((-4)/(np.pi * n ** 2)) * (np.pi*n*np.sin(np.pi * n) + np.cos(np.pi * n))


def bn_f(n):
    """
    Since g is an even function, by remark 1.2 b_n is 0.
    """
    return 0

n = 5
print(an_f(n)) # Convergent to 0
print(bn_f(n)) # Getting 0 as expected

0.050929581789406014
0


In [44]:
def a_n(f, n):
    # We used scipy.integrate to calculate the integral
    # scipy.integrate() returns a tuple,
    # where the first element holding the estimated value of the integral
    # and the second element holding an upper bound on the error
    subresult = integrate.quad(lambda x: f(x)*np.cos(n*x), -np.pi, np.pi)[0]
    return (1/(2*np.pi))*subresult

def b_n(f, n):
    # We used scipy.integrate to calculate the integral
    # scipy.integrate() returns a tuple,
    # where the first element holding the estimated value of the integral
    # and the second element holding an upper bound on the error
    subresult = integrate.quad(lambda x: f(x)*np.sin(n*x), -np.pi, np.pi)[0]
    return (1/(2*np.pi))*subresult

# def fourier_series(f, a_n, b_n):
#     return a_n(0) + 2 * sum_to_infinity()

# def sum_to_infinity(N=10000):
#     eps = 10**(-5)
#     i = 2
#     sum1 = sum(a_n(f, 1) * np.cos(x) + b_n(f, 1) * np.sin(x))
#     while abs(sum1) < eps and i < N:
#         sum1 += sum(a_n(f, i) * np.cos(i*x) + b_n(f, i) * np.sin(i*x))
#         i+=1

def f1(x):
    return np.sin(x)
n = 5
print(a_n(f1, n)) # 0 since sin(x) is an odd function
print(b_n(f1, n)) # Convergent to 0


0.0
1.3877787807814457e-17


Study of decay of the Fourier coefficients (the rate at which they go to 0)